In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "KaggleUsername"
os.environ['KAGGLE_KEY'] = "KaggleToekn"
!kaggle datasets download -d gpiosenka/100-bird-species
!unzip 100-bird-species


In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tqdm as tqdm
from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.utils import shuffle
from keras import utils

In [0]:
# creates a data generator object that transforms images
datagen = ImageDataGenerator(
  rotation_range=20,
  zoom_range=0.1,
  width_shift_range=0.2,
  height_shift_range=0.1,
  shear_range=0.15,
  horizontal_flip=True,
  fill_mode='nearest'
)


In [0]:
IMG_SIZE = 224
train_classes = os.listdir('./180/train')[0:30]
def extract_folder(folder_name,generate=False):
  labels = []
  data= []
  for index, type in tqdm.tqdm(enumerate(train_classes)):
    items = os.listdir('./180/{}/{}'.format(folder_name,type))
    i=0
    for bird in items:
      img = image.load_img('./180/{}/{}/{}'.format(folder_name,type,bird), 
                      target_size=(IMG_SIZE, IMG_SIZE))
      data.append(image.img_to_array(img))
      labels.append(index)
      if generate:  
        img_gen = image.img_to_array(img)
        img_gen = img_gen.reshape((1,) + img_gen.shape)
        for batch in datagen.flow(img_gen, save_prefix='test', save_format='jpeg'):        
          i += 1
          if i > 6:
              break
          data.append(image.img_to_array(img))
          labels.append(index)
  return np.array(data,dtype=np.float32)/255, np.array(labels)


In [0]:
train_x, train_y = extract_folder('train',True)
validation_x, validation_y = extract_folder('valid')


In [0]:
train_x, train_y = shuffle(train_x, train_y)
validation_x, validation_y = shuffle(validation_x, validation_y)

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [0]:
model = tf.keras.Sequential();
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(train_classes), activation='softmax'))
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [0]:
history = model.fit(train_x, train_y,
                    epochs=10,
                    validation_data=(validation_x,validation_y), shuffle=True)

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [0]:
from tensorflow.python.keras.preprocessing import image
test_image_path = './180/test/BLACK SWAN/1.jpg'
img = image.load_img(test_image_path, target_size=(224, 224))
img = np.array(image.img_to_array(img), dtype=np.float32)/255
data = img.reshape(1,IMG_SIZE, IMG_SIZE,3)
predict = model.predict(data)
print(predict)
plt.title(train_classes[np.argmax(predict)])
plt.imshow(img)
plt.show()